## Convolutional Autoencoder arquitecture optimization

Con lo concluído en el anterior notebook, nos ocupamos de buscar la arquitectura de nuestro CAE -número de capas convolucionales y densas, número de filtros, funciones de activación, etc.- que mejor performance tenga ante los datos generados en 1.0; estandarizados con nuestro modelo RobustScaler.

In [1]:
import pickle
import os

from keras.wrappers.scikit_learn import KerasRegressor
from matplotlib import pyplot as plt 
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import losses

from deep_scattering_models.data.create_data import load_data

from deep_scattering_models.features.preprocess_data import to_dB

from deep_scattering_models.models.convolutional_autoencoder import ConvAutoencoder

from deep_scattering_models.models.model_wrappers import build_cae_architecture

from deep_scattering_models.models.select_model import k_fold_cv, save_configuration

from deep_scattering_models.visualization.visualize import plot_polarization_signature, plot_history

Cargamos los datos crudos, para poder cargarlos en un pipline que los estandarice, pase a dB y les de forma para ingresar al CAE, en el contexto de una validación cruzada (cada uno de estos pasos debe realizarse en cada iteración, sin cruzar datos entre los folds de entrenamiento y testeo).

In [2]:
raw_data_filename = "raw/spm_signatures_no_noise"
raw_data = load_data(raw_data_filename)

In [3]:
data_dB = to_dB(raw_data)

In [4]:
INPUT_SHAPE = (45, 90, 1)

Vamos a realizar un Grid Search barriendo varias arquitecturas posibles. Para eso definimos una grilla o espacio de parámetros, sobre el cual podemos obtener un modelo para cada un de las combinaciones posibles de parámetros y evaluar su performance con nuestros datos. Para que la evaluación sea más robusta, vamos a usar K-fold Cross Validation.

Comenzamos definiendo nuestra grilla de parámetros. Vamos a dejar fijo el parámetro correspondiente a la dimensión del espacio latente, en número bajo para optimizar el resto de la arquitectura -acá suponemos que la performance del modelo va a mejorar para dimensiones más altas del espacio latente-.

In [5]:
# Check GPU's 
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Fijamos la dimensión del espacio latente
LATENT_DIMENTION = 3

# Configuracion de las capas convolucionales
cv_layers = [
    [(4, (3, 4), 2), (8, (3, 3), 1)],
    [(16, (3, 4), 2), (32, (3, 3), 1)],
    [(32, (5, 6), 2), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 2), (128, (3, 3), 1)],
    [(16, (3, 4), 2), (32, (3, 3), 2), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (2, 2), 2), (128, (3, 3), 1)],
    [(128, (3, 4), 2), (256, (3, 3), 2), (512, (3, 3), 1)],
    [(128, (3, 4), 2), (256, (3, 3), 1), (256, (3, 3), 1)],
    [(128, (3, 4), 2), (256, (2, 2), 1), (256, (1, 1), 1)], 
    [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 2), (128, (3, 3), 2), (256, (3, 3), 1)]
    ]

max_pooling = [True, False]

In [7]:
# Configuración de las capas densas
dense_layers = [
    (8,), (32,), (64,), (256,),
    (32, 16), (16, 16), (64, 32),
    (256, 128), (512, 256), (32, 32),
    (1024, 512), (512, 64), (256, 32),
    (64, 32, 16), (128, 64, 32), (256, 128, 64),
    (64, 32, 32), (32, 32, 16), (1024, 512, 128),
    (128, 64, 32, 16), (256, 128, 64, 32)
 ]

drop_out = [True, False]

Una vez definida la grilla sobre la cual se va a hacer una búsqueda de hiperparámetros, hacemos una busqueda iterando sobre cada una de las configuraciones de la grilla. Para cada configuración vamos a realizar una k-fold cross validation con k=5.

In [11]:
configurations_score = []
best_configuration = {'score' : 1e4}

for cv_layer in tqdm(cv_layers[-2:]):
    for dense_layer in dense_layers[-2:]:
        for pooling in max_pooling:
            for dropout in drop_out:
                # Configuración del modelo
                conv_configuration = dict(
                    layers_config=cv_layer,
                    max_pooling=pooling,
                    kernel_initializer= 'glorot_uniform'
                )

                dense_configuration = dict(
                    layers_units=dense_layer,
                    dropout=dropout
                )                

                configuration = {
                    'conv_layers_config': conv_configuration, 
                    'dense_layers_config': dense_configuration, 
                    'batch_size': 16
                }
                print(configuration)

                # k-fold cross validation
                k_fold_score = k_fold_cv(
                    data_dB, build_cae_architecture, configuration
                )
                
                # Actualizo los scores de las configuraciones
                configuration.update(k_fold_score)
                configurations_score.append(configuration)
                    
                if configuration['score'] < best_configuration['score']:
                    best_configuration = configuration 
         

  0%|          | 0/2 [00:00<?, ?it/s]

{'conv_layers_config': {'layers_config': [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1), (64, (3, 3), 1)], 'max_pooling': True}, 'dense_layers_config': {'layers_units': (128, 64, 32, 16), 'dropout': True}, 'batch_size': 16}


  0%|          | 0/2 [01:47<?, ?it/s]


ResourceExhaustedError: OOM when allocating tensor with shape[38912,38912] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

Visualizamos los puntajes de las distintas configuraciones ordenados por score

In [ ]:
df_scores = pd.DataFrame.from_records(configurations_score).sort_values(by='score')
df_scores

Guardo la arquitectura con mejor score

In [ ]:
save_configuration(
    best_configuration, 
    filename='model_architecture'
    )